# Importing packages and load CNN

In [ ]:
# # Load the extension
# %load_ext autoreload
# # Autoreload all modules
# %autoreload 2

In [ ]:
import numpy as np
import matplotlib.pyplot as plt

import imageio
import time
import os
import busworks

from pypylon import pylon
from matplotlib import cm

In [ ]:
import funcs

# Telling the notebook to make plots inline.
%matplotlib inline

plt.rc('text', usetex=True)
plt.rc('font', family='serif', size=12)

## Start camera and DAC

In [ ]:
# Create an instant camera object with the camera device found first
camera = pylon.InstantCamera(pylon.TlFactory.GetInstance().CreateFirstDevice())
print("Using Camera ", camera.GetDeviceInfo().GetModelName())

camera.Open()

# Sanity checks
print('Camera Auto Gain is :', camera.GainAuto())
if camera.GainAuto() == 'Off':
    print('Safe to go ahead with the code..')
else:
    print('***Warning: Camera Auto Gain is ON!***\nTurn it off using Pylon software.')

# Set exposure
camera.ExposureTimeAbs = funcs.Exposure

In [ ]:
# Allowed number of generations
NumGens = 10
# Number of images to be grabbed.
countOfImagesToGrab = funcs.pop_per_gen * funcs.N_CM_STEPS * (NumGens+2)

# Start the grabbing of c_countOfImagesToGrab images.
# The camera device is parameterized with a default configuration which
# sets up free-running continuous acquisition.
camera.StartGrabbingMax(countOfImagesToGrab)

In [ ]:
# Start DAC
start_channel = 1
bus = busworks.BusWorks_DAC()
bus.start()
print("DAC on IP: ", bus.address)

bus.read_voltages()

## Initial State

In [ ]:
Img_initial = funcs.Capture_image(camera, 500)
print('Initial condition: {}'.format(Img_initial.sum()/funcs.n_pixl**2))
plt.imshow(Img_initial, cmap=cm.binary_r)
plt.colorbar()

In [ ]:
# Initial condition of beam
current_beam_status = np.zeros(funcs.num_params)

## Alignment loop

In [ ]:
Range = funcs.Range_orig

In [ ]:
# Creating the initial population of deltas
new_pop_deltas = funcs.sample_d(Range, shape=funcs.pop_size, first_sample=True)
print("\nRange:", Range, "x waist")
# sess = tf.Session()

gen = 0
mode_stability = 0
mode_old = (50,50)
# dont_alter_z_cm = False
t0 = time.time()

# GA loop
for gen in range(NumGens):
    # Shrink range
    Range *= funcs.shrink_factor
    print("\n\n Gen: {} Range: {} x waist".format(gen+1, Range))
    # if dont_alter_z_cm:
    #     new_pop_deltas[:,-1] = 0.
    if gen == 0:
        current_beam_status, new_pop_deltas, fitness = funcs.calc_pop_fitness(current_beam_status, \
                                                                              new_pop_deltas, camera, bus, \
                                                                              only_offsprings=False)
    else:
        current_beam_status, new_pop_deltas, fitness = funcs.calc_pop_fitness(current_beam_status, \
                                                                              new_pop_deltas, camera, bus, \
                                                                              only_offsprings=True)
    # Selecting the best parents in the population for mating.
    current_beam_status, parents, parents_fitness, Img_is_saturated = \
            funcs.select_mating_pool(current_beam_status, new_pop_deltas, fitness, \
                                     funcs.num_parents_mating, t0, gen, camera, bus, funcs.BEST_IMG_OF_GEN, \
                                     show_the_best=True, save_best=True)
    # Generating next generation using crossover.
    offspring_crossover = funcs.crossover(parents, (funcs.pop_per_gen - funcs.num_parents_mating, \
                                                    funcs.num_params))
    # Adding some variations to the offsrping using mutation.
    offspring_mutation = funcs.mutation(current_beam_status, offspring_crossover, Range)
    # Creating the new population based on the parents and offspring.
    new_pop_deltas[:funcs.num_parents_mating, :] = parents
    new_pop_deltas[funcs.num_parents_mating:, :] = offspring_mutation
    fitness[:funcs.num_parents_mating] = parents_fitness
    fitness[funcs.num_parents_mating:] *= 0.
    # Show mode for best image in gen
    mode_new = funcs.Find_mode2(funcs.BEST_IMG_OF_GEN, separation1=funcs.SEPARATION, Width=funcs.WIDTH, \
                          thresh=funcs.THRESH, corner=0, show_fig=True, show_basis=True)
    print("Mode: ", mode_new)
    if mode_new == (0,0):
        mode_stability += 1
    else:
        mode_stability = 0
    # mode stability for 3 consecutive gens exits loop
    if mode_stability == 3:
        print('Fundamental mode identified! Exiting..')
        break

## Make gif of the images

In [ ]:
files = os.listdir(funcs.ImagesFolder)
files.sort()
images = []
for filename in files:
    images.append(imageio.imread(funcs.ImagesFolder+'/'+filename))
imageio.mimsave(funcs.ImagesFolder + '/movie.gif', images, duration=0.5)

# Simple lock

## jump to best alignment

In [ ]:
print('Best alignment: ', funcs.BEST_BEAM_STATUS, "; reward: ", funcs.BEST_REWARD)
funcs.Set_Voltage(np.append(funcs.BEST_BEAM_STATUS, 0), bus)
# Best alignment:  [-0.00016919 -0.00048279  0.00123649  0.00314689]
# Set_Voltage([8.71353902e-05, -3.69861200e-04, 1.58933381e-03, 2.61383266e-03, 0], bus)

## Get max power

In [ ]:
%matplotlib notebook

In [ ]:
show_fig = True

# P_max = 10000
P_max = funcs.BEST_REWARD/(2e4/funcs.n_pixl**2./funcs.Exposure) # inverted reward fn to get image power
lock_record = [0.]
P_record = [0.]
Zphi = 0.
direction = 1   # direction of movement

# Set exposure
funcs.Exposure = 150
camera.ExposureTimeAbs = funcs.Exposure

# get max power again
print("Max power: ", P_max)

for i in range(2000):
    tt1 = time.time()
    Img = funcs.Capture_image(camera, 500)
    P_new = Img.sum()
    # adjust z-step according to o/p power
    if P_new < P_max/100.:
        z_step = 5e-10
    else:
        z_step = 1e-11
    # if P_max exceeded, reset to new value
    if P_new > P_max:
        P_max = P_new
        print("Max power reset to : ", P_max)

## fringe lock loop

In [ ]:
# create fig obj
if show_fig:
    fig, (ax1, ax2) = plt.subplots(2,1, figsize=(8,6))

# lock loop
for i in range(10000):
    tt1 = time.time()
    Img = funcs.Capture_image(camera, 500)
    P_new = Img.sum()
    # adjust z-step according to o/p power
    if P_new < P_max/1000.:
        z_step = 1e-9
    else:
        z_step = 1e-12
    # maintaining FWHM position
    if P_new < P_max / 2.:
        # positive dir
        sign = 1
    else:
        # take an opposite step
        sign = -1
    # take a step
    Zphi += sign*direction*z_step
    # if hits end of z-range
    if (Zphi < 0) or (Zphi > funcs.Lambda):
        direction *= -1
        continue
    # cavity mirror scan step
    funcs.Set_Voltage(np.append(funcs.BEST_BEAM_STATUS, Zphi), bus)
    # record data
    lock_record.append(Zphi)
    P_record.append(P_new)
    # time step
    delta_t = time.time() - tt1
    # fig
    if show_fig and i%10==0:
        # update image
        ax1.clear()
        ax1.imshow(Img, cmap=cm.binary_r)
        ax2.clear()
        ax2.plot(np.arange(len(lock_record))*(delta_t), np.array(lock_record), 'r')
        ax2.set_xlabel('time (sec)')
        ax2.set_ylabel('$\Delta L$')
        fig.canvas.draw()

In [ ]:
# plt.plot(P_record)

# Stop Camera and DAC

In [ ]:
# Stop camera and DAC
camera.Close()
bus.stop()